In [1]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter

In [2]:
stopwords=['ଦେଇଛନ୍ତି', 'ଲେଖାଏଁ', 'ଜଣେ', 'ଏହା', 'ତେଣୁ', 'ମିଳିଥାଏ', 'ପାଇଁ', 'ନେଉଛନ୍ତି', 'ଯୋଗୁଁ', 'ଏପର୍ଯ୍ୟନ୍ତ', 'ଏଭଳି', 'କରୁଛି', 'ଓ', 'ଯାଏଁ','ହୋଇଛନ୍ତି', 'କି', 'କରାଯିବା', 'ପରେ', 'ଏହି', 'ଏବଂ', 'ଜଣ', 'ଥିବା','ହୋଇଥିଲା', 'ତେବେ', 'ଆଜି', 'ଜଣଙ୍କ', 'ଏଥି', 'ଗତ', 'ହୋଇଥିଲେ',
       'ଦିନ ଧରି', 'ହେଉଥିବା', 'ଦିନ ତଳେ', 'ଯୋଗେ', 'ବୋଲି', 'ଜଣାପଡ଼ିଛି',
       'ଦ୍ବାରା', 'କରି', 'ଯାଇ', 'ଏ ନେଇ', 'ଚାଲୁ', 'ରହିଛି', 'ତାରିଖ',
       'ମିଳିଛି', 'ବର୍ଷୀୟ', 'ଦିନ ତଳେ', 'ସହ', 'ଆସିଛନ୍ତି', 'ମଧ୍ୟ', 'କେଉଁ',
       'ହୋଇଯାଇଛି', 'ନେଇ ଯାଇଛି', 'କାମ', 'କରିଛି', 'ହେବା', 'ପରେ', 'ଏବେ',
       'ହୋଇଛି', 'ତେବେ', 'ରଖି', 'ନିଆଯାଇଥିବା', 'ପର୍ଯ୍ୟନ୍ତ', 'କରିଛନ୍ତି',
       'ଉପରେ', 'ଦେଲେ', 'ଥର', 'ଏବଂ', 'ଆଉ']

In [3]:
alphabets=['ଅ','ଆ','ଇ','ଈ','ଉ','ଊ','ଋ','ଌ','ଏ','ଐ','ଓ','ଔ','କ','ଖ','ଗ','ଘ','ଙ','ଚ','ଛ','ଜ','ଝ','ଞ','ଟ','ଠ','ଡ','ଢ','ଣ',
 'ତ','ଥ','ଦ','ଧ','ନ','ପ','ଫ','ବ','ଭ','ମ','ଯ','ର','ଲ','ଳ','ଵ','ଶ','ଷ','ସ','ହ','ଡ଼','ଢ଼','ୟ','ୠ','ୡ','ୱ']
matra=['ଁ','ଂ','ଃ','଼','ା','ି','ୀ','ୁ','ୂ','ୃ','ୄ','େ','ୈ','ୋ','ୌ','୍','ୢ','ୣ']

In [4]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq
def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)
        
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())

In [5]:
data=pd.read_csv('../odia.csv')

In [6]:
doc_words=[]
dict_global = {}
idx = 0
files_with_index = {}
for index in range(len(data)):
    fname=str(data.loc[index]['Chapter'])+"."+str(data.loc[index]['Verse'])
    sloka=data.loc[index]['Sloke']
    text=data.loc[index]['Commentary']
    text = remove_special_characters(text)
    words = text.split()
    words = [word for word in words if word not in stopwords]
    doc_words.append(words)
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = fname
    idx = idx + 1
    
unique_words_all = set(dict_global.keys())
    


In [7]:
tf={}
df={}
for i in unique_words_all:
    tf[i]={}
    df[i]=0   

In [8]:
idx=0
Ltot=0
Ld={}
for index in range(len(data)):
    text=data.loc[index]['Commentary']
    text = remove_special_characters(text)
    words = text.split()
    words = [word for word in words if word not in stopwords]
    Ld[idx]=len(words) #len of current doc
    Ltot=Ltot+len(words) #sum of lens of all the docs
    counter=Counter(words)
    for i in counter.keys():
        df[i]=df[i]+1
        tf[i][idx]=counter[i]   
    idx=idx+1

In [9]:
doc_norm={}
idx=0
for i in doc_words:
    l2=0
    for j in set(i):
        l2+=(i.count(j)*math.log(len(files_with_index)/df[j]))**2
    doc_norm[idx]=(math.sqrt(l2))
    idx+=1

In [10]:
with open('pickles/posting_list.pkl','wb') as file:
    pickle.dump(tf,file)
    file.close()
    
with open('pickles/df.pkl','wb') as file:
    pickle.dump(df,file)
    file.close()
    
with open('pickles/doc_len.pkl','wb') as file:
    pickle.dump(Ld,file)
    file.close()

In [11]:
a_file = open("pickles/file_idx.pkl", "wb")
pickle.dump(files_with_index, a_file)
a_file.close()
a_file = open("pickles/unique_words_all.pkl", "wb")
pickle.dump(unique_words_all , a_file)
a_file.close()

In [12]:
with open('pickles/doc_words.pkl','wb') as file:
    pickle.dump(doc_words,file)
    file.close()
    
with open('pickles/doc_norm.pkl','wb') as file:
    pickle.dump(doc_norm,file)
    file.close()

In [14]:
remove_special_characters(data.loc[45]['Commentary'])

'ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କରିବାକୁ ଯାଇ ଆମେ ସେମାନଙ୍କର ପରିଣାମ ସମ୍ବନ୍ଧୀୟ ବିଭିନ୍ନ ବିକଳ୍ପ ଉପରେ ବିଚାର କରିଥାଉ ଅର୍ଜୁନ ବିଚାର କରୁଛନ୍ତି ଯୁଦ୍ଧରେ କୌରବମାନଙ୍କୁ ପରାଜିତ କରିବା ଉଚିତ ନା କୌରବଙ୍କ ଦ୍ୱାରା ପରାଜିତ ହେବା ଉଚିତ ଉଭୟ ବିକଳ୍ପ ପରାଜୟ ପରି ହିଁ ପ୍ରତୀତ ହେଉଛି କାରଣ କୌରବମାନଙ୍କୁ ହତ୍ୟା କରି ସେ ଯଦି ଯୁଦ୍ଧରେ ଜୟ ଲାଭ କରନ୍ତି ତେବେ ତାଙ୍କର ବଞ୍ ଚବାର ଇଚ୍ଛା ରହିବ ନାହିଁ କିନ୍ତୁ ବାସ୍ତବରେ ଭୀଷ୍ମ ଦ୍ରୋଣ କୃପାଚାର୍ଯ୍ୟ ଇତ୍ୟାଦି ବିଧର୍ମୀ କୌରବମାନଙ୍କର ପକ୍ଷ ନେଇ ନିକୃଷ୍ଟ ଆଚରଣ ପ୍ରଦର୍ଶନ କରିଛନ୍ତି ଅର୍ଥକାମ ଶବ୍ଦଟି ଏଠାରେ ସେମାନଙ୍କ ପାଇଁ ବ୍ୟବହାର କରାଯାଇଛି ଯାହାର ଅର୍ଥ ବୈଭବ ଏବଂ ପଦବୀ ପ୍ରତି ଆସକ୍ତ ରହି ସେମାନେ ଦୁଷ୍ଟ ଦୁର୍ଯ୍ୟୋଧନଙ୍କର ପକ୍ଷ ସ୍ୱୀକାର କରିଥିଲେ ତେଣୁ ଯୁଦ୍ଧରେ ସେମାନଙ୍କୁ ହତ୍ୟା କରିବା ଏକ ସ୍ୱାଭାବିକ ପରିଣତି ଥିଲା ପ୍ରକୃତରେ ଯୁଦ୍ଧ ଶେଷ ହେବା ପରେ ଭୀଷ୍ମ ନିଜେ ସ୍ୱୀକାର କରିଥିଲେ ଯେ ଜଣେ ଶିକ୍ଷକ ଯଦି ନୀଚ କର୍ମ କରନ୍ତି ତେବେ ତାଙ୍କୁ ତ୍ୟାଗ କରାଯିବା ଉଚିତ୍ ଏଠାରେ ଭୀଷ୍ମଙ୍କ ବିଷୟରେ ସ୍ୱତନ୍ତ୍ର ବିଚାର ଆବଶ୍ୟକ ଶ୍ରୀମଦ୍ ଭାଗବଦମ୍ ଶ୍ଲୋକ ଅନୁସାରେ ସେ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ପରମ ଭକ୍ତ ଥିଲେ ସେ ଇନ୍ଦ୍ରିୟଜୟୀ ଥିଲେ ଏବଂ ନୈତିକତା ଓ ଉଦାରତାର ମୂର୍ତ୍ତିମାନ ସ୍ୱରୂପ ଥିଲେ ସେ ଚିରନ୍ତନ ସତ୍ୟର ଜ୍ଞାତା ଏବଂ ଜୀବନରେ ସତ୍ୟବାଦୀ ହେବା